In [ ]:
import requests
from bs4 import BeautifulSoup as bSoup
import re
import pandas as pd

<b>Once the file is uploaded and selected, the following block of will be inserted by IBM Watson Studio's "Insert to code" feature. More about this can be read at</b> https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/load-and-access-data.html.

In [ ]:
import types
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0

# Access credentials are marked as "**********" all through the notebook, please replace them as necessary
client_********** = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='**********',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_**********.get_object(Bucket='**********',Key='Gallary-Awards-gov2019.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)

In [ ]:
def listToString(l):
    str1 = ''
    for i in l:
        str1 += i    
    return str1

In [ ]:
df.rename(columns = {'_ - name': 'name', '_ - external_url': 'url'}, inplace = True)
df['award_year']=''
df['service_num']=''

In [ ]:
df.head()

In [ ]:
for index in df.index:
    url = df['url'][index]
    page = requests.get(url)
    pageHTML = bSoup(page.content, 'html.parser')
    #converted entire HTML code to string as there is not specific HTML class assigned for year of award
    pageString = str(pageHTML)
    yearLine = re.findall("<div class=\"col-xs-12 col-sm-6 col-md-6 lside\"\>Year of Award<\/div>\\n<div class=\"col-xs-12 col-sm-6 col-md-6\"\>.*<\/div>", pageString)
    yearLine = listToString(yearLine)
    #this step was not directly performed as there might be other four digit numbers on a page
    awardYear = re.findall("\d{4}", yearLine)
    if len(awardYear) == 0:
        awardYear = 0
    df.loc[index, 'award_year'] = awardYear
    serviceLine = re.findall("<div class=\"col-xs-12 col-sm-6 col-md-6 lside\"\>Service No.<\/div>\\n<div class=\"col-xs-12 col-sm-6 col-md-6\"\>.*<\/div>", pageString)
    serviceLine = listToString(serviceLine)
    #there is not consitent pattern for service number, so eliminated the known pattern of HTML tags around it
    serviceNum = serviceLine.replace('<div class="col-xs-12 col-sm-6 col-md-6 lside">Service No.</div>\n<div class="col-xs-12 col-sm-6 col-md-6">', '').replace('</div>', '')
    if len(serviceNum) == 0:
        serviceNum = 'NOT FOUND'
    df.loc[index, 'service_num'] = serviceNum

<b>More on exporting a DataFrame to cloud can be read at</b> https://www.ibm.com/support/pages/how-do-you-save-dataframe-watson-studio-project-notebook-asset-cloud-object-storage.

In [ ]:
from project_lib import Project
#second parameter is the project ID and third parameter is the access token
GAProject = Project(None, "**********","**********")
GAProject.save_data(file_name = "GA2019Updated-IN.csv", data = df.to_csv(index=False))